In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from operator import itemgetter
from nltk.tokenize import word_tokenize
import csv
import re
import string
import os
import fnmatch
import numpy as np
import pandas as pd
import sys
import scipy.sparse as sp
import ast
import cPickle
import datetime

mystops = [u'i',u'me',u'my',u'myself',u'we',u'our',u'ours',u'ourselves',u'you',
           u'your',u'yours',u'yourself',u'yourselves',u'he',
           u'him',u'his',u'himself',u'she',u'her',u'hers',u'herself',u'it',
           u'its',u'itself',u'they',u'them',u'their',u'theirs',u'themselves',
           u'what',u'which',u'who',u'whom',u'this',u'that',u'these',u'those',
           u'am',u'is',u'are',u'was',u'were',u'be',u'been',u'being',u'have',u'has',
           u'had',u'having',u'do',u'does',u'did',u'doing',u'a',u'an',u'the',u'and',u'but',
           u'if',u'or',u'because',u'as',u'until',u'while',u'of',u'at',u'by',u'for',u'with',
           u'about',u'against',u'between',u'into',u'through',u'during',u'before',u'after',u'above',
           u'below',u'to',u'from',u'up',u'down',u'in',u'out',u'on',u'off',u'over',u'under',
           u'again',u'further',u'then',u'once',u'here',u'there',u'when',u'where',u'why',u'how',
           u'all',u'any',u'both',u'each',u'few',u'more',u'most',u'other',u'some',u'such',u'no',
           u'nor',u'not',u'only',u'own',u'same',u'so',u'than',u'too',u'very',u's',u't',u'can',
           u'will',u'just',u'don',u'should',u'now']
### get current file path
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))

##set up folders
picklesdir = currentdir
if not os.path.exists(picklesdir):
        os.mkdir(picklesdir)
validPicklesdir = picklesdir+'/validPickles'
if not os.path.exists(validPicklesdir):
        os.mkdir(validPicklesdir)

### get lookup file
wDict = {}
execfile(str(currentdir)+"/abbr_lookup.py", wDict)

def getTSList(pickelspath = validPicklesdir):
    dates = []
    for file in sorted(os.listdir(pickelspath)):
        if fnmatch.fnmatch(file, '*.pkl'):
            dates.append(file.split('-')[1].split('.')[0])            
    # filter unique dates and sort them in ascending order
    return sorted(list(set(dates)))   

import cPickle
def pickle_model(filename,model):
    with open(str(picklesdir+'/'+filename) + '.pkl', 'wb') as fid:
        cPickle.dump(model, fid)
    print "Pickled Model at" + str(picklesdir)

def read_pickle(filename,pickelspath = validPicklesdir):
    with open(pickelspath + '/' + str(filename) + '.pkl', 'rb') as fid:
        gnb_loaded = cPickle.load(fid)
    return gnb_loaded


def load_pickles(TS):
    svmModelL1 = read_pickle('paycode_L1_SVM_classifier-'+TS)
    svmModelL2 = read_pickle('paycode_L2_SVM_classifier-'+TS)
    vectorizerL1 = read_pickle('paycode_L1_tfidf_vectorizer-'+TS)
    vectorizerL2 = read_pickle('paycode_L2_tfidf_vectorizer-'+TS)
    return (svmModelL1,svmModelL2,vectorizerL1,vectorizerL2)

def mark_pickle_TS_valid(TS):
    frompath = picklesdir+'/paycode_L1_SVM_classifier-'+TS+'.pkl'
    topath = validPicklesdir+'/paycode_L1_SVM_classifier-'+TS+'.pkl'
    os.rename(frompath,topath)
    
    frompath = picklesdir+'/paycode_L2_SVM_classifier-'+TS+'.pkl'
    topath = validPicklesdir+'/paycode_L2_SVM_classifier-'+TS+'.pkl'
    os.rename(frompath,topath)
    
    frompath = picklesdir+'/paycode_L1_tfidf_vectorizer-'+TS+'.pkl'
    topath = validPicklesdir+'/paycode_L1_tfidf_vectorizer-'+TS+'.pkl'
    os.rename(frompath,topath)
    
    frompath = picklesdir+'/paycode_L2_tfidf_vectorizer-'+TS+'.pkl'
    topath = validPicklesdir+'/paycode_L2_tfidf_vectorizer-'+TS+'.pkl'    
    os.rename(frompath,topath)

# get the last element, it will be the latest date as teh list is sorted
#pickles = load_pickles(getTSList(currentdir).pop())
pickles = load_pickles(getTSList().pop())
#####################################################################################################
## Text Pre-processing functions
def cleanText(s,keepPunct):
    import string
    from nltk.tokenize import word_tokenize
    punctuation = ''.join(list(set(string.punctuation) - set(keepPunct)))
    table = string.maketrans("","")
    s = s.translate(table,punctuation)
    s = ' '.join(''.join(i for i in token if not i.isdigit()) for token in word_tokenize(s))
    return s.strip()

def multiwordReplace(text, wDict):
    '''
    Replace words in a text that match a key in replace_dict
    with the associated value, return the modified text.
    Only whole words are replaced.
    '''
    rc = re.compile(r"[A-Za-z_]\w*")
    def translate(match):
        word = match.group(0)
        return wDict.get(word, word)
    return rc.sub(translate, text)

def cleanPayCd(pcname,pcdesc,keepPunct,wDict):
    #mystops = stopwords.words('english')
    paycd_text = pcdesc + ' ' + pcname
    prepText = ' '.join(token.lower() for token in word_tokenize(paycd_text) if token not in mystops)
    prepText = cleanText(prepText,keepPunct = keepPunct)
    prepText = ' '.join(multiwordReplace(t,wDict) for t in prepText.split(' '))
    return prepText

def preprocess(pcData,keepPunct=[],wrdDict=wDict):
    pcData['PAYCD_TEXT'] = pcData['PAYCD_SHRT_DSC'] + ' ' + pcData['PAYCD_NM']
    paycd_text = list(pcData['PAYCD_TEXT'])
    prepText = [' '.join(token.lower() for token in word_tokenize(text) if token not in mystops) 
                for text in paycd_text]
    prepText = [cleanText(text,keepPunct = keepPunct) for text in prepText]
    prepText = [' '.join(multiwordReplace(t,wrdDict) for t in text.split(' ')) for text in prepText]
    pcData.drop('PAYCD_TEXT',axis=1,inplace=True)
    pcData['PAYCD_TEXT'] = pd.Series(prepText, index=pcData.index)
    print "--------------------------------------------------------------------- \n"
    print "Pre-Processing Completed for " + str(pcData.shape[0]) + " PayCode records..."
    print "--------------------------------------------------------------------- \n"
    return pcData
####################################################################################################
### predict methods reading stored pickles of Models and Vectorizers
def predictL1(model,vectorizer,pcdesc,pcname,wDict,threshold=0.8):
    paycodeText= cleanPayCd(pcname,pcdesc,keepPunct=['-','$'],wDict=wDict)
    X_pred = vectorizer.transform(np.array([paycodeText]))
    pclasses = model.classes_
    y_predicted = model.predict(X_pred[0])[0]
    yprob = model.predict_proba(X_pred[0])[0]
    yprob = [round(x,3) for x in yprob]
    prediction = [(x,y) for (y,x) in sorted(zip(yprob,pclasses),reverse=True)]
    prediction = [pred[1][[x[0] for x in pred[1]].index(pred[0])] for pred in zip([y_predicted],[prediction])]
    if prediction[0][1] <= threshold:
        return (("Other",1),paycodeText)
    else:
        return (prediction[0],paycodeText)


def predictL2(modelL2,vectorizer,modelL1,paycodeL1):
    ### filter where predictions are necessary
    if paycodeL1[0][0] == "Other":
        prediction = ("Other",1)
        return prediction
    elif paycodeL1[0][0] == "Regular":
        prediction = ("Regular",1)
        return prediction
    elif paycodeL1[0][0] == "Premium Time":
        prediction = ("Doubletime",1)
        return prediction
    else:
        L2Original = vectorizer.transform(np.array([paycodeL1[1]]))
        L2Classes = ["Absence","Overtime"]
        dummyL1 = sp.csr_matrix(pd.get_dummies(pd.Series(list(L2Classes)))[paycodeL1[0][0]].as_matrix())
        X_test = sp.hstack((L2Original,dummyL1),format='csr')
        yprob = [round(x,3) for x in modelL2.predict_proba(X_test[0])[0]]
        prediction = [(x,y) for (y,x) in sorted(zip(yprob,modelL2.classes_),reverse=True)]
        prediction = [pred[1][[x[0] for x in pred[1]].index(pred[0])] for pred in zip([modelL2.predict(X_test[0])[0]],[prediction])]
        return prediction[0]

### un-pickle saved components
#svmModelL1 = read_pickle('paycode_L1_SVM_classifier')
#svmModelL2 = read_pickle('paycode_L2_SVM_classifier')
#vectorizerL1 = read_pickle('paycode_L1_tfidf_vectorizer')
#vectorizerL2 = read_pickle('paycode_L2_tfidf_vectorizer')

### wrapper function for prediction (both L1 and L2)
def classifyPC(pcname,pcdesc):
    L1pred = predictL1(model=pickles[0],vectorizer=pickles[2],pcdesc=pcdesc,pcname=pcname,wDict=wDict)
    L2pred = predictL2(modelL2=pickles[1],vectorizer=pickles[3],modelL1=pickles[0],paycodeL1=L1pred)
    return {'paycodeGroupCode':L1pred[0][0],'paycodeSubGroupCode':L2pred[0]}


### wrapper function to predict for a DataFrame of PayCodes --uses classifyPC defined above
def batchPCpredict(store_location,filename,input_path=currentdir):
    if filename.endswith('.csv'):
        data = pd.read_csv(input_path + '/' + filename,header=0)
    else:
        data = pd.read_csv(input_path + '/' + filename + '.csv',header=0)
    data['groupCode']=data.apply(lambda x:classifyPC(str(x['name']),
                                                  str(x['shortDescription']))['paycodeGroupCode'],axis=1)
    data['subGroupCode']=data.apply(lambda x:classifyPC(str(x['name']),
                                                  str(x['shortDescription']))['paycodeSubGroupCode'],axis=1)
    if store_location == '':
        store_location = input_path
    if filename.endswith('.csv'):
        data.to_csv(store_location + '/' + str(filename)[:-4] + '_predicted.csv',index=False)
        print  str(filename)[:-4] + '_predicted.csv' + ' saved at ' + store_location + '/'
    else:
        data.to_csv(store_location + '/' + str(filename) + '_predicted.csv',index=False)
        print  str(filename) + '_predicted.csv' + ' saved at ' + store_location + '/'
